In [25]:
from prettytable import PrettyTable #for creating tables

#from scipy.stats import chisquare

import csv #for reading csv file
from math import ceil

# Netflix
Netflix je v poslední době velmi populární streamovací platforma a tak jsem vybral její téma pro vypracování statistické práce. Provedu nad daty dvě statistické metody. Pomocí první ověřím podle státní příslušnosti uživatelů ze vzorku, jak moc dobře náhodný je náš vzorek dat. Ve druhém bodě pak otestuji závislost mezi věkem a velikostí předplatného.


## Data
Data jsou pořízené ze stránky kaggle.com. Vzorek obashuje 2500 řádků, kde každý obashuje popořadě následující sloupce: User ID (identifikátor uživatele), Subscription Type (typ předplatného), Monthly Revenue (defacto se jedná o cenu předplatného v dané zemi v dolarech), Join Date (datum vytvoření účtu uživatele), Last Payment Date (poslední datum, kdy byla provedena platba), Country (země původu), Age (věk), Gender (pohlaví), Device (nejčastěji používané zařízení uživatele), Plan Duration (na jakou dobu má uživatel předplaceno). 

Načtení a vizualizace dat:

In [19]:
tab = PrettyTable()
data = None

with open("data/netflix-userbase.csv", newline="") as file:
    dict_reader = csv.DictReader(file)
    
    data = list(dict_reader)
    
    tab.field_names = dict_reader.fieldnames
    
    for row in dict_reader:  
        tab.add_row(
            [
                row["User ID"],
                row["Subscription Type"],
                row["Monthly Revenue"],
                row["Join Date"],
                row["Last Payment Date"],
                row["Country"],
                row["Age"],
                row["Gender"],
                row["Device"],
                row["Plan Duration"]
            ]
        )

print(tab.get_string(
    start=0, #sets boundaries for first and last rows we want to print out
    end=5,
    fields=["User ID", "Subscription Type", "Monthly Revenue", "Country", "Age"] #projection of some of the columns
))

+---------+-------------------+-----------------+---------+-----+
| User ID | Subscription Type | Monthly Revenue | Country | Age |
+---------+-------------------+-----------------+---------+-----+
+---------+-------------------+-----------------+---------+-----+


Do notebooku se na šířku nevejdou všechny sloupce z datového souboru, takže jsem vyprojektoval pouze ty, které nás budou zajímat pro budoucí testy.

### Aktuální počty obyvatel
Dodatečně pro test dobré shody budeme potřebovat data o aktuálních počtech obyvatel, které jsem použil z wikipedie, konkrétně: https://cs.wikipedia.org/wiki/Seznam_států_světa_podle_počtu_obyvatel. Níže je seznam zemí, který nás bude zajímat, jelikož se nachází ve vzorku.

In [27]:
countries = []

for row in data:
    country = row["Country"]
    
    if country not in countries:
        countries.append(country)

print("Countries from our data:")
for i in range(len(countries)):
    print(countries[i], end=", ") if i < len(countries) - 1 else print(countries[i])

Countries from our data:
United States, Canada, United Kingdom, Australia, Germany, France, Brazil, Mexico, Spain, Italy


A pro tyto země získáme následující pravděpodobnosti, že uživatel je dané národnosti vzhledem k celkovému počtu obyvatel:

- United states: 4.19%
- Canada: 0.48%
- United Kingdom: 0.85%
- Australia: 0.32%
- Germany: 1.05%
- France: 0.86%
- Brazil: 2.7%
- Mexico: 1.65%
- Spain: 0.55%
- Italy: 0.75%

## Test náhodnosti dat
Pro tento test použijeme údaje o zemi původu uživatelů. Spočteme jaká je očekávaná četnost podle světových dat, že člověk pochází z dané země. A porovnáme s pozorovanou četností z našeho vzorku. Pro následné porovnání výsledků použijeme test dobré shody. Nulová hypotéza tedy bude, že máme dobře spravedlivý vzorek, nebo-li tedy vzorek obsahuje přímo úměrný počet obyvatel z jednotlivých zemí vzhledem k jejich celosvětové populaci. Alternativní hypotézou bude tvrzení říkající, že tato vlastnost ve vzorku naopak neplatí.

Spočtěme nejprve tedy očekávané četnosti, podle procentuálních dat z wikipedie s tím, že budeme brát horní celou část výsledků (součet pravděpodobností nedává dohromady 1, takže přidání nějakých četností navíc noevlivní tolik výsledek a chceme počítat s celými čísly):

In [28]:
n = 2500
expected = [
    ceil(n*0.0419),
    ceil(n*0.0048),
    ceil(n*0.0085),
    ceil(n*0.0032),
    ceil(n*0.0105),
    ceil(n*0.0086),
    ceil(n*0.027),
    ceil(n*0.0165),
    ceil(n*0.0055),
    ceil(n*0.0075)
]

print(expected) #TODO: print histogram

[105, 12, 22, 8, 27, 22, 68, 42, 14, 19]
